In [ ]:
!pip install datasets

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from tqdm import tqdm
from datasets import load_dataset
from tokenizers import SentencePieceBPETokenizer
from transformers import LlamaTokenizerFast

In [23]:
import pandas as pd

# Load the Excel file
file_path = '/content/ibibio.xlsx'  # Replace with your file path
df = pd.read_excel(file_path)

# Extract the 'transcriptions' column
transcriptions = df['transcriptions']

# Save the transcriptions to a text file
output_file_path = 'transcriptions.txt'  # Replace with your desired output file path
transcriptions.to_csv(output_file_path, index=False, header=False)


In [24]:
path = "/content/transcriptions.txt"

In [25]:
def get_training_corpus(dataset, batch_size=300):
    """
    Generator function to yield batches of transcriptions from the dataset.
    """
    for i in range(0, len(dataset["train"]), batch_size):
        yield dataset["train"][i: i + batch_size]["transcriptions"]

def main():
    # Dataset name
    dataset_name = "aidystark/ibibio"

    # Load dataset
    dataset = load_dataset(dataset_name)

    # Create and train the tokenizer using the training corpus generator
    tokenizer = SentencePieceBPETokenizer()
    training_corpus = get_training_corpus(dataset)

    tokenizer.train(
        files=path,
        vocab_size=32000,
        min_frequency=2,
        special_tokens=["<pad>", "<s>", "</s>", "<unk>"],
        show_progress=True
    )

    # Save the tokenizer
    tokenizer.save("tokenizer.json")
    print("Tokenizer saved to tokenizer.json")

if __name__ == "__main__":
    main()


Using the latest cached version of the dataset since aidystark/ibibio couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/aidystark___ibibio/default/0.0.0/0ab6193d4c6b2d4b4900c4038f2d63439ae714d4 (last modified on Mon May 27 11:28:12 2024).


Tokenizer saved to tokenizer.json
